# Import utilities

In [1]:
from community_utils_3layers import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

import igraph as ig

D:\Programs\Anaconda\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
%load_ext autoreload
%autoreload 2

----

# Read the files

If you do not have any partitions yet, just read the graph, normalize it in 'Normalization' and move to the section 'Define the best partitions for all the methods'.

##### Read the graph itself

We use only 'value' attribute.

In [3]:
G = nx.read_gml('data\datasets\polblogs\polblogs.gml').to_undirected()

'''Change ureadble URL for PolBlogs'''
mapping = {'charlineandjamie.com/dotnetweb01a/blogdisplay.aspx?logname=jamie&logcatid=48': 'charlineandjamie.com'}
nx.relabel_nodes(G, mapping, copy=False)

str_graph, att_graph = nx.Graph(), nx.Graph()
for edge in G.edges():
    str_graph.add_edge(edge[0], edge[1], weight=1)
    if G.node[edge[0]]['value'] == G.node[edge[1]]['value']:
        att_graph.add_edge(edge[0], edge[1], weight=1)
        
'''There might be nodes without attributes. We should add these nodes to attributive layer'''
att_graph.add_nodes_from(str_graph)

##### Read the partitions data if it already exists
'pol_ef_partit.txt' -- the name of the file with the partition for the EF-method \
'pol_sf_partit.txt' -- the name of the file with the partition for the SF-method \
'pol_lf_partit.txt' -- the name of the file with the partition for the LF-method \
'polblogs.gml' -- the name of the file with the graph itself

In [ ]:
partition = {name: read_raw_file(file_name) 
             for name, file_name in zip(['ef', 'sf' ,'lf'], ['data/partitions/polblogs/pol_ef_partit.txt', 
                                                             'data/partitions/polblogs/pol_sf_partit.txt', 
                                                             'data/partitions/polblogs/pol_lf_partit.txt'])}

Read the best partitions for the structural and attributive components that are used in LF \
'pol_par_first' -- the name of the file with the partition for the structural part \
'pol_par_second' -- the name of the file with the partition for the attributive part

In [ ]:
partition_phase1 = {name: read_raw_file(file_name) for name, file_name 
                    in zip(['str', 'att'], ['data/partitions/polblogs/pol_par_first.txt', 
                                            'data/partitions/polblogs/pol_par_second.txt'])}

----

# Normalization

In [4]:
str_graph = normalize_to_fixed_weight_nx(str_graph, 1, show=True)
att_graph = normalize_to_fixed_weight_nx(att_graph, 1, show=True)

The whole weight has been equalt to 16718 . Now it is equal to 1.0000000000000544
The whole weight has been equalt to 15143 . Now it is equal to 1.0000000000002434


In [5]:
graph = {'str': str_graph, 'att': att_graph}
graph_names = graph.keys()

##### SAVE

In [ ]:
save('data/datasets/polblogs/pol_att_graph.txt', list(att_graph.edges(data=True)))
save('data/datasets/polblogs/pol_str_graph.txt', list(str_graph.edges(data=True)))

----

# Define the best partitions for all the methods (Louvain)
If they do not yet exist, run that part and then run the whole notebook from the very beggining except this section.

In [10]:
simulations_number = 10
alphas_grid = [(val, 1-val) for val in np.linspace(0, 1, 21)]
alpha_order = {key: val for val, key in enumerate(graph_names)}
node_list = graph[take_first(graph)].nodes()

In [11]:
sf_partit = SF_louvain(graph, simulations_number, alphas_grid, alpha_order)

In [12]:
ef_partit = EF_louvain(graph, simulations_number, alphas_grid, alpha_order)

In [13]:
lf_partit, best_partit = LF_louvain(graph, simulations_number, alphas_grid, alpha_order)

Run this too if you have just defined the best partitions

In [14]:
partition = {'sf': sf_partit, 'ef': ef_partit, 'lf': lf_partit}

##### SAVE

In [36]:
save('data/partitions/polblogs/pol_sf_partit.txt', sf_partit)
save('data/partitions/polblogs/pol_ef_partit.txt', ef_partit)
save('data/partitions/polblogs/pol_lf_partit.txt', lf_partit)
save('data/partitions/polblogs/pol_par_first.txt', str_partit)
save('data/partitions/polblogs/pol_par_second.txt', att_partit)

NameError: name 'str_partit' is not defined

----

# Composite modularity of the methods in terms of different objective functions

In [15]:
Q_sf = {method: compute_experiment('sf', partition[method], graph, node_list, alpha_order)
        for method in ['sf', 'ef', 'lf']}

Q_ef = {method: compute_experiment('ef', partition[method], graph, node_list, alpha_order)
        for method in ['sf', 'ef', 'lf']}

short_graph_layers_lf = {item[0]: late_fusion_transient_absorption(item[1], best_partit[item[0]]) 
                             for item in graph.items()}
Q_lf = {method: compute_experiment('lf', partition[method], short_graph_layers_lf, node_list, alpha_order)
        for method in ['sf', 'ef', 'lf']}

##### SAVE

In [ ]:
save('data/metrics/polblogs/pol_Q_sf.txt', Q_sf)
save('data/metrics/polblogs/pol_Q_ef.txt', Q_ef)
save('data/metrics/polblogs/pol_Q_lf.txt', Q_lf)

##### PLOT

In [16]:
plot_dots_2d(Q_sf, '$Q_{SF}$', 'data/plots/polblogs/pol_sf.pdf')

In [17]:
plot_dots_2d(Q_ef, '$Q_{EF}$', 'data/plots/polblogs/pol_ef.pdf')

In [18]:
plot_dots_2d(Q_lf, '$Q_{LF}$', 'data/plots/polblogs/pol_lf.pdf')

----

## Delta

In [19]:
pattern = {'mean': None, 'std': None}
delta = {name: pattern for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    delta[name] = delta_experiment(partition[name], str_graph, att_graph)

##### SAVE

In [ ]:
save('data/metrics/polblogs/pol_delta.txt', delta)

##### PLOT

In [20]:
plot_dots_2d(delta, '$\Delta$', 'data/plots/polblogs/pol_delta.pdf')

----

## Theta

In [21]:
theta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    theta[name] = theta_experiment(partition[name], best_partit, graph)

##### SAVE

In [ ]:
save('data/metrics/polblogs/theta.txt', theta)

##### PLOT

In [22]:
plot_dots_2d(theta, '$\Theta$', 'data/plots/polblogs/pol_theta.pdf')

## Delta (G', C)

In [23]:
new_graph = {key: late_fusion_transient_absorption(graph[key], best_partit[key]) for key in graph.keys()}
delta_on_new_graph = {name: delta_experiment(partition[name], new_graph['str'], new_graph['att']) 
                      for name in ['sf', 'ef', 'lf']}

##### PLOT

In [24]:
plot_dots_2d(delta_on_new_graph, '$\Delta`$', 'data/plots/polblogs/pol_delta_new.pdf')

# Leiden

In [25]:
simulations_number = 10
alpha_order = {key: val for val, key in enumerate(graph_names)}

In [26]:
node_list = take_first(graph.values()).nodes()
    
alphas_grid = [(val, 1-val) for val in np.linspace(0, 1, 21)]
    #  Calculate partitions
sf_partit = SF_leiden(graph, simulations_number, alphas_grid, alpha_order)
ef_partit = EF_leiden(graph, simulations_number, alphas_grid, alpha_order)
lf_partit, best_partit = LF_leiden(graph, simulations_number, alphas_grid, alpha_order)
    
#     best_partit = {k: val for k, val in zip(new_graph_nx.keys(), [str_partit, att_partit])}
partition = {'sf': sf_partit, 'ef': ef_partit, 'lf': lf_partit}
    
Q_sf = {method: compute_experiment('sf', partition[method], graph, node_list, alpha_order) for method in ['sf', 'ef', 'lf']}
Q_ef = {method: compute_experiment('ef', partition[method], graph, node_list, alpha_order) for method in ['sf', 'ef', 'lf']}

short_graph_layers_lf = {item[0]: late_fusion_transient_absorption(item[1], best_partit[item[0]]) 
                         for item in graph.items()}
Q_lf = {method: compute_experiment('lf', partition[method], short_graph_layers_lf, node_list, alpha_order) 
        for method in ['sf', 'ef', 'lf']}

##### PLOT

In [27]:
plot_dots_2d(Q_sf, '$Q_{SF}$', 'data/plots/polblogs/pol_sf_leiden.pdf')

In [28]:
plot_dots_2d(Q_ef, '$Q_{EF}$', 'data/plots/polblogs/pol_ef_leiden.pdf')

In [29]:
plot_dots_2d(Q_lf, '$Q_{LF}$', 'data/plots/polblogs/pol_lf_leiden.pdf')

### Delta

In [30]:
delta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    delta[name] = delta_experiment(partition[name], graph['str'], graph['att'])

##### PLOT

In [31]:
plot_dots_2d(delta, '$\Delta$', 'data/plots/polblogs/pol_delta_leiden.pdf')

### Theta

In [32]:
theta = {name: None for name in['sf', 'ef', 'lf']}
for name in ['sf', 'ef', 'lf']:
    theta[name] = theta_experiment(partition[name], best_partit, graph)

##### PLOT

In [33]:
plot_dots_2d(theta, '$\Theta$', 'data/plots/polblogs/pol_theta_leiden.pdf')

### Delta'

In [34]:
new_graph = {key: late_fusion_transient_absorption(graph[key], best_partit[key]) for key in graph.keys()}
delta_on_new_graph = {name: delta_experiment(partition[name], short_graph_layers_lf['str'], short_graph_layers_lf['att']) 
                      for name in ['sf', 'ef', 'lf']}

##### PLOT

In [35]:
plot_dots_2d(delta_on_new_graph, '$\Delta`$', 'data/plots/polblogs/pol_delta_new_leiden.pdf')